In [131]:
import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [133]:
# Define the ROIs (already provided)
roi_for_brightness = ee.Geometry.Polygon(
    [[163.05418845709113, -77.62217141675877], 
    [163.04800864752082, -77.61908009873933], 
    [163.0795943408802, -77.61422077772508], 
    [163.08302756841925, -77.61569349746868],
    [163.09813376959113, -77.60950691594756], 
    [163.1407057910755, -77.60567560295046], 
    [163.15924521978644, -77.60228539245952], 
    [163.1736647754505, -77.60140083961204], 
    [163.17435142095832, -77.5996315475443],
    [163.18259116705207, -77.59786200695194],
    [163.19769736822394, -77.60125340809783], 
    [163.21417686041144, -77.60184312380088], 
    [163.2505690723255, -77.59874680831578], 
    [163.25812217291144, -77.60036878277327],
    [163.25400229986457, -77.60302247238047],
    [163.25468894537238, -77.60493867830638], 
    [163.25880881841925, -77.60670722499943], 
    [163.2334029346302, -77.60788611813062], 
    [163.21967002447394, -77.60655985559598],
    [163.20799705084113, -77.60980158403946],
    [163.21280356939582, -77.61156944773495], 
    [163.19357749517707, -77.61716604483516], 
    [163.18121787603644, -77.6155462332516], 
    [163.15237876470832, -77.61687154915111],
    [163.14894553716925, -77.62025783330262],
    [163.12010642584113, -77.62231860342517], 
    [163.10637351568488, -77.62217141675877], 
    [163.0795943408802, -77.61893287416414], 
    [163.06448813970832, -77.61849119009841],
    [163.06723472173957, -77.62099386140082],
    [163.06448813970832, -77.6226129715892],
    [163.05418845709113, -77.62217141675877]]
)

In [86]:
# Define ROI for soil
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.07800, -77.625340])

# Pull image
soil1 = ee.Image('LANDSAT/LC08/C02/T2_TOA/LC08_055116_20231205').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

# Clip image down to ROI for soil
soil = soil1.clip(roi_soil)

# Reduce down to mean for endmembers
soil_mean_defined = soil.reduceRegion(ee.Reducer.mean()).values()

In [87]:
# Function to compute brightness and average
def select_brightest_pixels_and_average(image, roi, n=1):
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)
    sorted_points = sampled_points.sort('sum', False)
    top_n_brightest_points = sorted_points.limit(n)

    def compute_mean_of_window(feature):
        coords = feature.geometry().coordinates()
        lon = ee.Number(coords.get(0))
        lat = ee.Number(coords.get(1))
        buffer = ee.Geometry.Point([lon, lat]).buffer(90, ee.ErrorMargin(1))  # 3x3 pixels (90m buffer)
        window_mean = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=buffer,
            scale=30
        )
        return ee.Feature(None, window_mean)

    means = top_n_brightest_points.map(compute_mean_of_window)

    # Extract values for all bands
    def extract_band_values(feature):
        band_values = image.bandNames().map(lambda band: feature.get(band))
        return ee.Feature(None, {'values': band_values})

    return means.map(extract_band_values)



In [88]:
def define_endmembers(image, roi, roi_soil, n=1):
    brightest_pixel_means = select_brightest_pixels_and_average(image, roi, n)
    soil_mean = image.reduceRegion(ee.Reducer.mean(), geometry=roi_soil, scale=30).values()

    # ensure endmembers are in the correct format
    endmembers = [ee.List(brightest_pixel_means), ee.List(soil_mean)]
    return endmembers

In [89]:
def perform_unmixing(image, endmembers):
    brightest_pixel_means = endmembers[0]
    soil_mean = endmembers[1]
    return image.unmix([brightest_pixel_means, soil_mean])

In [90]:
#endmembers = define_endmembers(soil1, roi_for_brightness, roi_soil, n=1)
#print(endmembers)
result_image = perform_unmixing(soil1, endmembers)
print(result_image)


ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.unmix",
    "arguments": {
      "endmembers": {
        "arrayValue": {
          "values": [
            {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_1_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Feature",
                          "arguments": {
                            "metadata": {
                              "dictionaryValue": {
                                "values": {
                                  "values": {
                                    "functionInvocationValue": {
                                      "functionName": "List.map",
          

In [77]:
print(result_image.getInfo())

EEException: Image.unmix, argument 'endmembers': Invalid type.
Expected type: List<List<Float>>.
Actual type: List<Object>.
Actual value: [<FeatureCollection>, [0.20610506406852178, 0.1811014584132603, 0.17480088344642095, 0.15912956637995582, 0.17721728554793767, 0.16994245563234603, 0.8713829432215009]]

In [18]:
print(soil_mean_defined.getInfo())

[0.20610506406852178, 0.1811014584132603, 0.17480088344642095, 0.15912956637995582, 0.17721728554793767, 0.16994245563234603, 0.8713829432215009]


In [80]:
def select_brightest_pixels_and_average(image, roi, n=1):
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)
    print("Sampled points size:", sampled_points.size().getInfo())
    
    sorted_points = sampled_points.sort('sum', False)
    print("Sorted points size:", sorted_points.size().getInfo())
    
    if sorted_points.size().getInfo() == 0:
        print("No points found after sorting.")
        return None

    top_n_brightest_points = sorted_points.limit(n)
    
    def compute_mean_of_window(feature):
        coords = feature.geometry().coordinates()
        lon = ee.Number(coords.get(0))
        lat = ee.Number(coords.get(1))
        print("Point coordinates:", lon.getInfo(), lat.getInfo())
        
        buffer = ee.Geometry.Point([lon, lat]).buffer(900, ee.ErrorMargin(1))  # 3x3 pixels (90m buffer)
        print("Buffered geometry:", buffer.getInfo())
        
        window_mean = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=buffer,
            scale=30
        )
        print("Window mean:", window_mean.getInfo())
        return ee.Feature(None, window_mean)

    means = top_n_brightest_points.map(compute_mean_of_window)
    means_size = means.size()
    print("Number of mean values computed:", means_size)

    if means_size == 0:
        print("No brightest pixel means found.")
        return None

    # Extract band values as a list
    def extract_band_values(feature):
        band_values = image.bandNames().map(lambda band: feature.get(band))
        return ee.List(band_values)

    band_values_list = means.map(extract_band_values).flatten()
    print("Band values list:", band_values_list.getInfo())
    return band_values_list


In [209]:

    # Compute brightness
brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    
    # Sample points within the ROI, including brightness values
sampled_points = brightness.addBands(image).sample(
        region=roi_for_brightness, 
        scale=30, 
        numPixels=1000
    )
    
    # Sort points by brightness in descending order
sorted_points = sampled_points.sort('sum', False)
    
    # Take the top N brightest points
top_n_points = sorted_points.limit(10)
    
    # Buffer each point by 90 meters (3x3 pixel equivalent for Landsat)
buffered_points = top_n_points.map(
        lambda feature: ee.Feature(
            feature.geometry().buffer(90), feature.toDictionary()
        )
    )
    
    # Combine buffered regions into a single geometry
combined_geometry = buffered_points.geometry()
    
    # Create a mask using the combined geometry
mask = ee.Image(1).clip(combined_geometry)
    
    # Apply the mask to the original image
clipped_image = image.updateMask(mask)
    



In [212]:
Map = geemap.Map(zoom=10, center=[-77.616808, 163.077952])
Map.addLayer(soil1)
#Map.addLayer(clipped_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, "Clipped Brightest Pixels with Buffer")
Map.addLayer(top_n_points, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, "Clipped Brightest Pixels with Buffer")
Map

EEException: Image.visualize: No band named 'B4'. Available band names: [vis-red, vis-green, vis-blue].

In [202]:
test = clip_to_brightest_pixels(soil1, roi_for_brightness, n=1)

In [203]:
print(test.getInfo())

EEException: Image.clip: The geometry for image clipping must not be empty.

In [206]:
image = ee.Image('LANDSAT/LC08/C02/T2_TOA/LC08_055116_20231205').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

In [198]:
brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    
    # Sample points within the ROI, including brightness values
sampled_points = brightness.addBands(image).sample(
        region=roi_for_brightness, 
        scale=30, 
        numPixels=1000
    )
    
print(sampled_points.size().getInfo())
    # Sort points by brightness in descending order
sorted_points = sampled_points.sort('sum', False)
print(sorted_points.size().getInfo())
    
    # Take the top N brightest points
top_n_points = sorted_points.limit(13)
print(top_n_points.size().getInfo())
    
    # Buffer each point by 90 meters (3x3 pixel equivalent for Landsat)
buffered_points = top_n_points.map(
        lambda feature: ee.Feature(
            feature.geometry().buffer(90), feature.toDictionary()
        )
    )

print(buffered_points.size().getInfo())
    
    # Combine buffered regions into a single geometry
combined_geometry = buffered_points.geometry()
    
    # Create a mask using the combined geometry
mask = ee.Image(1).clip(combined_geometry)
    
    # Apply the mask to the original image
clipped_image = image.updateMask(mask)

975
975
13
13


In [205]:
Map = geemap.Map(zoom=10, center=[-77.616808, 163.077952])
Map.addLayer(soil1)
Map.addLayer(clipped_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, "Clipped Brightest Pixels with Buffer")
Map

EEException: Image.clip: The geometry for image clipping must not be empty.